In [111]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import sympy as sp
import handcalcs.render
import forallpeople as si
import matplotlib.pyplot as plt
%matplotlib inline
from math import tan, sin,cos, pi, radians, sqrt, trunc
from IPython.display import display, Markdown
si.environment('structural', top_level=True)

<h2>Column verification</h2>

<h3>Compartment: Entrance and Lounge</h3>

<img src= "critical_column.png" style="width: 600px;"/>

<h4>Loads</h4>

In [112]:
%%render param 2

F_load  = 7395.08*kN
e_load = 192.60*kN*m/F_load

<IPython.core.display.Latex object>

<h4>Column parameters</h4>

In [113]:
%%render param 2

L = 11.4*m
b = 660*mm
h = 660*mm
c = 40*mm
f_ck20 = 55*MPa
E_c20 = 36000*MPa
phi_s = 20*mm
f_s20 = 500*MPa
E_s20 = 200*GPa


<IPython.core.display.Latex object>

From CONFIRE

<img src="confire_column4.png" width=400>

In [114]:
%%render param 2

T_max = (549*dC)
xi_CMH = 1
eta_H = 0.9438
T_cold = (552*dC) 
xi_CMC = 0.9922
eta_C = 0.8669

<IPython.core.display.Latex object>

In [115]:
cte = pd.read_excel('strength-reduction-constants.xlsx')

In [116]:
def material_constants(mat_index):
    k = cte['k '][mat_index]
    T1 = cte['T1 '][mat_index]*dC
    T2 = cte['T2 '][mat_index]*dC
    T8 = cte['T8 '][mat_index]*dC
    T64 = cte['T64'][mat_index]*dC
    return k, T1, T2, T8, T64



In [117]:
from handcalcs import handcalc
@handcalc(jupyter_display = True)
def material_deterioration(T,k,T1,T2,T8,T64):
    """
    Calculate the deterioration of a material.
    """
    xi = k + (1-k)/(1+T/T1+(T/T2)**2+(T/T8)**8+(T/T64)**64)
    return xi


Degradation factors for steel reinforcement

HOT

In [118]:
k, T1, T2, T8, T64 = material_constants(0)
xi_sH = material_deterioration(T_max,k,T1,T2,T8,T64)


<IPython.core.display.Latex object>

In [119]:
%%render
xi_sH

<IPython.core.display.Latex object>

COLD

In [120]:
k, T1, T2, T8, T64 = material_constants(2)
xi_sC = material_deterioration(T_cold,k,T1,T2,T8,T64)

<IPython.core.display.Latex object>

In [121]:
%%render
xi_sC

<IPython.core.display.Latex object>

**Before fire**

In [122]:
%%render long 2

A_s = 8*pi*phi_s**2/4 #Rebar area
F_su20 = A_s*f_s20 #Rebar strength
I_s = A_s*(h/2-(c+phi_s/2))**2 #Moment of inertia of steel bars
F_sE20 = pi**2*E_s20*I_s/L**2 #Euler resistance

<IPython.core.display.Latex object>

In [123]:
%%render long 2

A_c20 = b*h #Concrete area
I_c20 = b*h**3/12 #Moment of inertia of concrete
F_c20 = A_c20*f_ck20 #Strength of concrete section
F_cE20 = pi**2*I_c20*E_c20/L**2 #Euler resistance
F_R20 = (1/(F_c20+F_su20)+1/(F_cE20+F_sE20))**-1 #Rankine resistance

<IPython.core.display.Latex object>

Check on eccentricity and uncracked capacity

No thermal contribution to the eccentricy

In [124]:
e_20 = e_load

In [125]:
if e_20 < b/6: display(Markdown('Since $e_{20} <  b/6$, no tensile stress occur'))
else: display(Markdown('Tensile stresses will occur'))

Since $e_{20} <  b/6$, no tensile stress occur

In [126]:
%%render 2

F_ucr20 = F_R20*(1-2*e_20/b)
## Utilization factor
mu = F_load/F_ucr20

<IPython.core.display.Latex object>

**HOT - During fire**

Steel yield strength and Young modulus reduction

In [127]:
%%render 2

F_suH = xi_sH*F_su20
F_sEH = xi_sH*F_sE20

<IPython.core.display.Latex object>

Reduced cross-section

In [128]:
%%render 2

b_H = b*eta_H**(4/3)
#The next line is here only in case we change the section to a rectangular one. Otherwise, if it is squared the b_H will be the same as h_H
h_H = h-b*(1-eta_H**(4/3))
A_cH = b_H*h_H

<IPython.core.display.Latex object>

Reduced strength

In [129]:
%%render 2
F_cH = A_cH*f_ck20*xi_CMH 

<IPython.core.display.Latex object>

Reduced stiffness

In [130]:
%%render 2

E_cH = E_c20*xi_CMH**2
I_cH = b_H*h_H**3/12
F_cEH = pi**2*I_cH*E_cH/L**2

<IPython.core.display.Latex object>

Rankine resistance

In [131]:
%%render 2

F_RH = (1/(F_cH+F_suH)+1/(F_cEH+F_sEH))**-1

<IPython.core.display.Latex object>

Temperature at the edge

In [132]:
%%render 2

a_cH = b/2-b_H/2
T_cH = (603*dC)

<IPython.core.display.Latex object>

In [133]:
%%render 2

AE_cH = A_cH*E_cH
EI_cH = I_cH*E_cH
AE_sH = A_s*E_s20*xi_sH
EI_sH = I_s*E_s20*xi_sH


<IPython.core.display.Latex object>

Concrete stresses at the edges of the reduced cross-section

In [134]:
%%render long 2

sigma_c0H = F_load*E_cH/(AE_cH+AE_sH)
Delta_sigma_cH = F_load*e_load*E_cH/(EI_cH+EI_sH)*b_H/2


<IPython.core.display.Latex object>

In [135]:
%%render 2
sigma_c1H = sigma_c0H + Delta_sigma_cH
sigma_c2H = sigma_c0H - Delta_sigma_cH

<IPython.core.display.Latex object>

Transient factors

In [136]:
%%render
k_tr10H = max(1 - 2.35*sigma_c1H/f_ck20,0)
k_tr20H = max(1 - 2.35*sigma_c2H/f_ck20,0)

<IPython.core.display.Latex object>

Thermal deflection

In [137]:
%%render 2
e_thH = 1.1*10**-5/K*T_cH*(k_tr20H-k_tr10H)/b_H*L**2/8
e_H = e_load + e_thH

<IPython.core.display.Latex object>

In [138]:
if e_H < b_H/6: display(Markdown('Since $e_{H} <  b_{H}/6$, no tensile stress occur'))
else: display(Markdown('Tensile stresses will occur'))

Since $e_{H} <  b_{H}/6$, no tensile stress occur

In [139]:
%%render 2
F_ucrH = F_RH*(1-2*e_H/b_H)
## Utilization factor
mu_H = F_load/F_ucrH

<IPython.core.display.Latex object>

**COLD - After fire**

In [140]:
%%render 2

F_suC = F_su20
F_sEC = F_sE20

<IPython.core.display.Latex object>

Over-reinforcment check

In [141]:
%%render 2

b_C = b*eta_C**(4/3)
#The next line is here only in case we change the section to a rectangular one. Otherwise, if it is squared the b_H will be the same as h_H
h_C = h-b*(1-eta_C**(4/3))
A_cC = b_C*h_C

<IPython.core.display.Latex object>

Reduced strength

In [142]:
%%render 2
F_cC = A_cC*f_ck20*xi_CMC 

<IPython.core.display.Latex object>

Reduced stiffness

In [143]:
%%render 2

E_cC = E_c20*xi_CMC**2
I_cC = b_C*h_C**3/12
F_cEC = pi**2*I_cC*E_cC/L**2

<IPython.core.display.Latex object>

Rankine resistance

In [144]:
%%render 2

F_RC = (1/(F_cC+F_suC)+1/(F_cEC+F_sEC))**-1

<IPython.core.display.Latex object>

In [145]:
%%render 2
a_cC = b/2-b_C/2
e_C = e_load

<IPython.core.display.Latex object>

In [146]:
if e_C < b_C/6: display(Markdown('Since $e_{C} <  b_{C}/6$, no tensile stress occur'))
else: display(Markdown('Tensile stresses will occur'))

Since $e_{C} <  b_{C}/6$, no tensile stress occur

In [147]:
%%render 2

F_ucrC = F_RC*(1-2*e_C/b_C)
## Utilization factor
mu_C = F_load/F_ucrC

<IPython.core.display.Latex object>